002 Introduction to Trading


In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ta
import optuna
from pathlib import Path
import matplotlib.pyplot as plt
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator
from dataclasses import dataclass


In [22]:
# Ruta al CSV 
CSV_PATH = "Binance_BTCUSDT_1h.csv"

# CryptoDataDownload pone la primera línea como URL, por eso skiprows=1
data = pd.read_csv(CSV_PATH, skiprows=1).copy()
data.columns = [c.strip().replace(" ", "_") for c in data.columns]
data = data.rename(columns={"Date":"dt", "tradecount":"Trades"})

# Asegurar datetime UTC y orden temporal ascendente
data["dt"] = pd.to_datetime(data["dt"], utc=True, errors="coerce")
data = data.dropna(subset=["dt"]).sort_values("dt").reset_index(drop=True)

# Columnas mínimas esperadas
req_cols = ["dt","Open","High","Low","Close"]
assert all(c in data.columns for c in req_cols), "Faltan columnas OHLC en el CSV."

print("Rango temporal:", data["dt"].iloc[0], "→", data["dt"].iloc[-1])
print("Filas:", len(data))
data.head(3)


Rango temporal: 2017-08-17 04:00:00+00:00 → 2025-09-22 23:00:00+00:00
Filas: 70187


,Unix,dt,Symbol,Open,High,Low,Close,Volume_BTC,Volume_USDT,Trades
0,1502942400000,2017-08-17 04:00:00+00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,202366.138393,171
1,1502946000000,2017-08-17 05:00:00+00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,100304.823567,102
2,1502949600000,2017-08-17 06:00:00+00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,31282.312670,36


In [23]:
def add_indicators(df, ema_fast=12, ema_slow=26, rsi_window=14,
                   macd_fast=12, macd_slow=26, macd_signal=9):
    out = df.copy()
    out["ema_fast"] = EMAIndicator(close=out["Close"], window=ema_fast).ema_indicator()
    out["ema_slow"] = EMAIndicator(close=out["Close"], window=ema_slow).ema_indicator()
    out["rsi"]      = RSIIndicator(close=out["Close"], window=rsi_window).rsi()
    macd = MACD(close=out["Close"], window_fast=macd_fast, window_slow=macd_slow, window_sign=macd_signal)
    out["macd_line"]   = macd.macd()
    out["macd_signal"] = macd.macd_signal()
    return out



In [42]:
# === # === Señales 2-de-3 (versión selectiva) ===
def add_signals(df, rsi_upper=60, rsi_lower=40, macd_buffer=10.0):
    """
    1) EMAfast >= EMAslow  / <= (tendencia)
    2) RSI > rsi_upper / < rsi_lower (momento)
    3) MACD_line - MACD_signal > +buffer / < -buffer (confirmación)
    Confirmación 2-de-3. shift(1) para evitar look-ahead. Limpieza de NaNs.
    """
    out = df.copy()

    s1_long  = (out["ema_fast"] >= out["ema_slow"]).astype(int)
    s1_short = (out["ema_fast"] <= out["ema_slow"]).astype(int)

    s2_long  = (out["rsi"] >  rsi_upper).astype(int)
    s2_short = (out["rsi"] <  rsi_lower).astype(int)

    spread   = out["macd_line"] - out["macd_signal"]
    s3_long  = (spread >  macd_buffer).astype(int)
    s3_short = (spread < -macd_buffer).astype(int)

    votes_long  = s1_long + s2_long + s3_long
    votes_short = s1_short + s2_short + s3_short

    out["long_signal"]  = (votes_long  >= 2).shift(1).fillna(False).astype(bool)
    out["short_signal"] = (votes_short >= 2).shift(1).fillna(False).astype(bool)

    out = out.dropna(subset=["ema_fast","ema_slow","rsi","macd_line","macd_signal"]).reset_index(drop=True)
    return out


In [36]:
# === Backtest CORREGIDO: flujos de caja + MTM + margen sin apalancamiento ===
from dataclasses import dataclass
import numpy as np
import pandas as pd

@dataclass
class BTConfig:
    fee: float = 0.00125      # 0.125% por giro
    n_shares: float = 0.1     # tamaño por operación (p.ej., 0.1 BTC)
    stop_loss: float = 0.03   # 3%
    take_profit: float = 0.05 # 5%
    capital0: float = 10_000  # USDT

def backtest(df: pd.DataFrame, cfg: BTConfig):
    req = ["dt","Open","High","Low","Close","long_signal","short_signal"]
    for c in req:
        if c not in df.columns:
            raise ValueError(f"Falta columna requerida: {c}")

    d = df.reset_index(drop=True).copy()

    cash = float(cfg.capital0)
    position = 0.0           # +n_shares (long), -n_shares (short), 0 (flat)
    entry_price = None

    equity = []
    trade_pnls = []

    def can_open_long(price: float) -> bool:
        costo_total = cfg.n_shares * price * (1.0 + cfg.fee)
        return cash >= costo_total

    def can_open_short(price: float) -> bool:
        # exigimos margen en efectivo equivalente al costo de recompra (no apalancamiento)
        costo_recompra = cfg.n_shares * price * (1.0 + cfg.fee)
        return cash >= costo_recompra

    for _, row in d.iterrows():
        price = float(row["Close"])

        # --------- SALIDAS ----------
        if position != 0 and entry_price is not None:
            if position > 0:  # LONG
                hit_sl = price <= entry_price * (1 - cfg.stop_loss)
                hit_tp = price >= entry_price * (1 + cfg.take_profit)
                flip   = bool(row["short_signal"])

                if hit_sl or hit_tp or flip:
                    # vender para cerrar: entra efectivo (menos fee)
                    efectivo = position * price * (1.0 - cfg.fee)
                    pnl = (price - entry_price) * position
                    cash += efectivo
                    trade_pnls.append(pnl - position * price * cfg.fee)
                    position, entry_price = 0.0, None

            else:  # SHORT
                hit_sl = price >= entry_price * (1 + cfg.stop_loss)
                hit_tp = price <= entry_price * (1 - cfg.take_profit)
                flip   = bool(row["long_signal"])

                if hit_sl or hit_tp or flip:
                    # recomprar: sale efectivo (más fee)
                    efectivo = abs(position) * price * (1.0 + cfg.fee)
                    pnl = (entry_price - price) * abs(position)
                    cash -= efectivo
                    trade_pnls.append(pnl - abs(position) * price * cfg.fee)
                    position, entry_price = 0.0, None

        # --------- ENTRADAS ----------
        if position == 0:
            if row["long_signal"] and not row["short_signal"]:
                if can_open_long(price):
                    # comprar: sale efectivo (incluye fee)
                    efectivo = cfg.n_shares * price * (1.0 + cfg.fee)
                    cash -= efectivo
                    position = +cfg.n_shares
                    entry_price = price

            elif row["short_signal"] and not row["long_signal"]:
                if can_open_short(price):
                    # vender en corto: entra efectivo (menos fee)
                    efectivo = cfg.n_shares * price * (1.0 - cfg.fee)
                    cash += efectivo
                    position = -cfg.n_shares
                    entry_price = price

        # --------- VALORACIÓN MTM ----------
        mtm = cash + position * price
        equity.append(mtm)

    out = d.copy()
    out["equity"] = equity
    out["ret"] = out["equity"].pct_change().replace([np.inf,-np.inf], np.nan).fillna(0.0)
    out.attrs["trade_pnls"] = trade_pnls
    return out


In [37]:
from dataclasses import dataclass


@dataclass
class BTConfig:
    fee: float = 0.00125     # 0.125% por operación
    n_shares: float = 1.0
    stop_loss: float = 0.03
    take_profit: float = 0.05
    capital0: float = 10_000

def backtest(df: pd.DataFrame, cfg: BTConfig):
    req = ["dt","Open","High","Low","Close","long_signal","short_signal"]
    for c in req:
        if c not in df.columns:
            raise ValueError(f"Falta columna requerida: {c}")

    d = df.reset_index(drop=True).copy()
    cash = cfg.capital0
    position = 0.0
    entry_price = None
    equity, trade_pnls = [], []

    for _, row in d.iterrows():
        price = row["Close"]

        # --- Salidas ---
        if position != 0 and entry_price is not None:
            if position > 0:
                hit_sl = price <= entry_price * (1 - cfg.stop_loss)
                hit_tp = price >= entry_price * (1 + cfg.take_profit)
                flip   = row["short_signal"]
                if hit_sl or hit_tp or flip:
                    pnl = (price - entry_price) * abs(position)
                    cash += pnl
                    cash -= abs(position) * price * cfg.fee
                    trade_pnls.append(pnl - abs(position)*price*cfg.fee)
                    position, entry_price = 0.0, None
            else:
                hit_sl = price >= entry_price * (1 + cfg.stop_loss)
                hit_tp = price <= entry_price * (1 - cfg.take_profit)
                flip   = row["long_signal"]
                if hit_sl or hit_tp or flip:
                    pnl = (entry_price - price) * abs(position)
                    cash += pnl
                    cash -= abs(position) * price * cfg.fee
                    trade_pnls.append(pnl - abs(position)*price*cfg.fee)
                    position, entry_price = 0.0, None

        # --- Entradas ---
        if position == 0:
            if row["long_signal"] and not row["short_signal"]:
                position = +cfg.n_shares; entry_price = price
                cash -= abs(position) * price * cfg.fee
            elif row["short_signal"] and not row["long_signal"]:
                position = -cfg.n_shares; entry_price = price
                cash -= abs(position) * price * cfg.fee

        # --- Equity MTM ---
        mtm = cash
        if position != 0 and entry_price is not None:
            mtm += (price - entry_price) * abs(position) if position > 0 else (entry_price - price) * abs(position)
        equity.append(mtm)

    out = d.copy()
    out["equity"] = equity
    out["ret"] = out["equity"].pct_change().fillna(0.0)
    out.attrs["trade_pnls"] = trade_pnls
    return out

# Ejecutar
cfg = BTConfig()
bt = backtest(tmp, cfg)   # usa 'tmp' que ya trae indicadores y señales
print("Trades ejecutados:", len(bt.attrs.get("trade_pnls", [])))
bt[["dt","Close","equity","ret"]].head()


Trades ejecutados: 2915


,dt,Close,equity,ret
0,2017-08-18 13:00:00+00:00,4293.09,10000.000000,0.000000
1,2017-08-18 14:00:00+00:00,4259.40,10000.000000,0.000000
2,2017-08-18 15:00:00+00:00,4236.89,10000.000000,0.000000
3,2017-08-18 16:00:00+00:00,4250.34,9994.687075,-0.000531
4,2017-08-18 17:00:00+00:00,4193.35,10051.677075,0.005702


In [39]:
HOURS_PER_YEAR = 24*365

def _clean(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce").astype(float)
    return s.replace([np.inf, -np.inf], np.nan).dropna()

def max_drawdown(equity: pd.Series) -> float:
    """Máx Drawdown como número NEGATIVO acotado a -1.0 ≤ MDD ≤ 0.0; devolver en valor POSITIVO para reportes."""
    eq = _clean(equity)
    if eq.empty:
        return 0.0
    roll_max = eq.cummax()
    dd = eq/roll_max - 1.0
    dd = dd.clip(lower=-1.0, upper=0.0)       # evita valores “>100%”
    return float(dd.min())                     # negativo (ej. -0.25)

def cagr(equity: pd.Series, periods_per_year: int = HOURS_PER_YEAR) -> float:
    """CAGR robusto: si la equity es ≤0 en algún punto, devolver 0 (no definido)."""
    eq = _clean(equity)
    if len(eq) < 2:
        return 0.0
    if (eq <= 0).any():                        # evita potencias de negativos
        return 0.0
    start, end = float(eq.iloc[0]), float(eq.iloc[-1])
    years = len(eq)/periods_per_year
    if years <= 0:
        return 0.0
    return float((end/start)**(1/years) - 1.0)

def calmar(equity: pd.Series) -> float:
    """Calmar = CAGR / |MDD|; si MDD==0 o equity no válida, devuelve 0."""
    eq = _clean(equity)
    if len(eq) < 2:
        return 0.0
    if (eq <= 0).any():
        return 0.0
    mdd = abs(max_drawdown(eq))
    if mdd == 0:
        return 0.0
    return float(cagr(eq) / mdd)



In [41]:
eq   = bt["equity"]
rets = bt["equity"].pct_change()

metrics = {
    "Sharpe" : round(sharpe(rets), 4),
    "Sortino": round(sortino(rets), 4),
    "Calmar" : round(calmar(eq), 4),
    "MaxDD"  : round(abs(max_drawdown(eq)), 4),  # positivo para el reporte
    "WinRate": round(win_rate(bt.attrs.get("trade_pnls", [])), 4),
    "Trades" : len(bt.attrs.get("trade_pnls", [])),
}
metrics


{'Sharpe': 0.5838,
 'Sortino': 2.4286,
 'Calmar': 0.0,
 'MaxDD': 1.0,
 'WinRate': 0.3461,
 'Trades': 2915}